In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt

# 🟡 مرحله اول: دریافت Access Token با clientId و clientSecret
client_id = "CLIENT_ID_خودت_اینجا"
client_secret = "CLIENT_SECRET_خودت_اینجا"

auth_payload = {
    "grantType": "ClientCredentials",
    "clientId": client_id,
    "clientSecret": client_secret
}

auth_response = requests.post(
    "https://api.topstepx.com/api/security/token",
    json=auth_payload,
    headers={"Content-Type": "application/json"}
)

if auth_response.status_code == 200:
    access_token = auth_response.json()["accessToken"]
    print("✅ Access Token دریافت شد")
else:
    print("❌ خطا در دریافت توکن:", auth_response.status_code)
    print(auth_response.text)
    raise SystemExit

# 🟡 مرحله دوم: گرفتن کندل‌های MNQU25
url = "https://api.topstepx.com/api/History/retrieveBars"
payload = {
    "contractId": "CON.F.US.MNQ.U25",
    "live": True,
    "startTime": "2025-06-20T00:00:00Z",
    "endTime": "2025-06-25T00:00:00Z",
    "unit": 2,
    "unitNumber": 5,
    "limit": 100,
    "includePartialBar": False
}

headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json",
    "accept": "application/json"
}

response = requests.post(url, json=payload, headers=headers)
if response.status_code == 200:
    bars = response.json()
    print(bars.keys())  # بررسی کلیدها
    df = pd.DataFrame(bars["bars"])
    df['timestamp'] = pd.to_datetime(df['startTime'])
    print(df[['timestamp', 'open', 'high', 'low', 'close']].tail())

    # رسم چارت قیمت
    plt.figure(figsize=(12, 5))
    plt.plot(df['timestamp'], df['close'], label='Close')
    plt.title("MNQU25 - 5 Minute Chart")
    plt.xlabel("Time")
    plt.ylabel("Price")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()
else:
    print("❌ خطا در گرفتن داده:", response.status_code)
    print(response.text)